# Sumador Draper con QFT

In [1]:
from pytket import Circuit
from pytket.utils import Graph
from pytket.backends.ibm import AerBackend, AerStateBackend
from qiskit.visualization import plot_bloch_multivector

In [2]:
def qft_rotations(circuit, n):
    """Performs qft on the first n qubits in circuit (without swaps)"""
    if n == 0:
        return circuit
    n -= 1
    circuit.H(n)   # anhadimos la puerta H
    for qubit in range(n):
        circuit.CRz((1/(2**(n-qubit))), qubit, n) # pasamos el numero, el qubit de control
                                            # y luego el objetivo.
    qft_rotations(circuit, n)

In [3]:
def adder(circuit, n):    
    circuit.CRz(1/2**(0), n  ,0)
    circuit.CRz(1/2**(1), n+1,0)
    circuit.CRz(1/2**(2), n+2,0)
    circuit.CRz(1/2**(3), n+3,0)
    
    circuit.CRz(1/2**(0), n+1,1)
    circuit.CRz(1/2**(1), n+2,1)
    circuit.CRz(1/2**(2), n+3,1)
   
    circuit.CRz(1/2**(0), n+2,2)
    circuit.CRz(1/2**(1), n+3,2)
  
    circuit.CRz(1/2**(0), n+3,3)
    circuit.add_barrier([0,1,2,3])

In [4]:
def swap_registers(circuit, n):
    for qubit in range(n//2): # tenemos que hacer el swap para queden bien 
        circuit.SWAP(qubit, n-qubit-1)
    return circuit

def qft(circuit, n):
    """QFT on the first n qubits in circuit"""
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [5]:
def qft_rotationsInverse(circuit):
    circuit.SWAP(0,3)
    circuit.SWAP(1,2)

    circuit.H(0)
    
    circuit.CRz(-0.5, 0, 1)
    circuit.H(1) 

    circuit.add_barrier([0, 1, 2, 3])
    circuit.CRz(-0.25, 0, 2)
    circuit.CRz(-0.5, 1, 2)
    circuit.H(2) 

    circuit.add_barrier([0, 1, 2, 3])
    circuit.CRz(-0.125, 0, 3)
    circuit.CRz(-0.25, 1, 3)
    circuit.CRz(-0.5, 2, 3) 
    circuit.H(3) 

In [6]:
n = 8
nA = 4
circ = Circuit(8)
#Nuestro numero A seria |1100> (12)
circ.X(0)
circ.X(1)
#nuestro numeroB seria |0001> (1)
circ.X(7)
qft(circ,nA)
adder(circ, nA)
qft_rotationsInverse(circ)

In [7]:
circ.measure_all()

[X q[0]; X q[1]; H q[3]; X q[7]; CRz(0.125*PI) q[0], q[3]; CRz(0.25*PI) q[1], q[3]; CRz(0.5*PI) q[2], q[3]; H q[2]; CRz(0.25*PI) q[0], q[2]; CRz(0.5*PI) q[1], q[2]; H q[1]; CRz(0.5*PI) q[0], q[1]; H q[0]; SWAP q[1], q[2]; SWAP q[0], q[3]; CRz(1*PI) q[4], q[0]; Measure q[4] --> c[4]; CRz(0.5*PI) q[5], q[0]; CRz(0.25*PI) q[6], q[0]; CRz(1*PI) q[5], q[1]; Measure q[5] --> c[5]; CRz(0.125*PI) q[7], q[0]; CRz(0.5*PI) q[6], q[1]; CRz(0.25*PI) q[7], q[1]; CRz(1*PI) q[6], q[2]; Measure q[6] --> c[6]; CRz(0.5*PI) q[7], q[2]; CRz(1*PI) q[7], q[3]; Measure q[7] --> c[7]; Barrier q[0], q[1], q[2], q[3]; SWAP q[0], q[3]; SWAP q[1], q[2]; H q[0]; CRz(3.5*PI) q[0], q[1]; H q[1]; Barrier q[0], q[1], q[2], q[3]; CRz(3.75*PI) q[0], q[2]; CRz(3.5*PI) q[1], q[2]; H q[2]; Barrier q[0], q[1], q[2], q[3]; CRz(3.875*PI) q[0], q[3]; Measure q[0] --> c[0]; CRz(3.75*PI) q[1], q[3]; Measure q[1] --> c[1]; CRz(3.5*PI) q[2], q[3]; Measure q[2] --> c[2]; H q[3]; Measure q[3] --> c[3]; ]

In [8]:
print(circ.get_commands())

[X q[0];, X q[1];, H q[3];, X q[7];, CRz(0.125*PI) q[0], q[3];, CRz(0.25*PI) q[1], q[3];, CRz(0.5*PI) q[2], q[3];, H q[2];, CRz(0.25*PI) q[0], q[2];, CRz(0.5*PI) q[1], q[2];, H q[1];, CRz(0.5*PI) q[0], q[1];, H q[0];, SWAP q[1], q[2];, SWAP q[0], q[3];, CRz(1*PI) q[4], q[0];, Measure q[4] --> c[4];, CRz(0.5*PI) q[5], q[0];, CRz(0.25*PI) q[6], q[0];, CRz(1*PI) q[5], q[1];, Measure q[5] --> c[5];, CRz(0.125*PI) q[7], q[0];, CRz(0.5*PI) q[6], q[1];, CRz(0.25*PI) q[7], q[1];, CRz(1*PI) q[6], q[2];, Measure q[6] --> c[6];, CRz(0.5*PI) q[7], q[2];, CRz(1*PI) q[7], q[3];, Measure q[7] --> c[7];, Barrier q[0], q[1], q[2], q[3];, SWAP q[0], q[3];, SWAP q[1], q[2];, H q[0];, CRz(3.5*PI) q[0], q[1];, H q[1];, Barrier q[0], q[1], q[2], q[3];, CRz(3.75*PI) q[0], q[2];, CRz(3.5*PI) q[1], q[2];, H q[2];, Barrier q[0], q[1], q[2], q[3];, CRz(3.875*PI) q[0], q[3];, Measure q[0] --> c[0];, CRz(3.75*PI) q[1], q[3];, Measure q[1] --> c[1];, CRz(3.5*PI) q[2], q[3];, Measure q[2] --> c[2];, H q[3];, Measure

In [9]:
b = AerBackend()
b.compile_circuit(circ)
handle = b.process_circuit(circ, 1000)
result = b.get_result(handle)
counts = result.get_counts()
print(counts)

Counter({(0, 0, 1, 0, 0, 0, 0, 1): 1000})


Como vemos, los 4 bits mas significactivos, nos da la suma de nuestro dos numeros, A+B = 4, pero de mode inversa.